<a href="https://colab.research.google.com/github/yuji-sgs/web-optimization/blob/main/cahpter1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1章: A/Bテストからはじめよう: ベイズ統計による仮説検定入門

In [3]:
# 必要なモジュールの読み込み
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

plt.style.use('grayscale')
plt.style.use('seaborn-whitegrid')
np.random.seed(0)

<ipython-input-3-973520a4796b>:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')
